In [3]:
import warnings
import pandas as pd
import asyncio
from rag_model import generate_chain_output
import os
from dotenv import load_dotenv
load_dotenv()
warnings.filterwarnings("ignore")


In [2]:
test_data = pd.read_pickle("/home/halil/Desktop/MeETU/RAG_Model/ragasio/controlled_test_data.pkl")

In [ ]:
def get_response_for_test_data(row):
    response, context = generate_chain_output(row)
    context = [doc.page_content for doc in context]
    # print(str(response)+ "\n\n")
    # print(str(context)+"\n\n")
    # print("-------")
    return response, context

In [70]:
test_data["response"], test_data["retrieved_contexts"] = test_data["user_input"].apply(get_response_for_test_data)

ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 124997b3-2cff-4396-bd74-6a64173cf932)')

In [5]:
test_subset = test_data.head()

In [6]:
test_subset["response"], test_subset["retrieved_contexts"] = zip(*test_subset["user_input"].apply(get_response_for_test_data))

PC Rooms Regulations are a set of rules that govern the use of personal computers in the university's PC rooms. These regulations are designed to ensure that students, faculty, and staff use the PC rooms in a responsible and respectful manner, while also providing a comfortable and productive environment for everyone.

The regulations cover various aspects, such as:

*   Computer usage and maintenance
*   Internet and network usage
*   Printing and copying
*   Noise levels and quiet hours
*   Personal belongings and security
*   Respect for others and university property

By following these regulations, students and faculty can enjoy a positive and productive experience in the PC rooms, while also contributing to a safe and respectful environment for everyone.

If you have any questions or concerns about the PC Rooms Regulations, please visit the university's website or contact the relevant authorities.

Would you like to know more about METU's dormitory services or any other aspect of

In [7]:
test_subset

,user_input,reference_contexts,reference,response,retrieved_contexts
0,How has the Turkish Standards Institution cont...,[General Information: There are 19 dormitory b...,The Office of Dormitories obtained the TSE-EN ...,PC Rooms Regulations are a set of rules that g...,[FAQ Answer: PC Rooms which are operated by ...
1,How does the Rectorate influence the dormitory...,[General Information: There are 19 dormitory b...,"Every year, the Executive Board of Dormitories...",To re-enroll to an undergraduate program at ME...,[Registration Procedures Question: What are th...
2,What are the operting hurs for transport?,[Transportation: There are buses and minibuses...,There are buses and minibuses operating betwee...,Hello! I'd be happy to help you with your ques...,[FAQ Answer: Brief Information on IT Service...
3,What hours buses run from city center to dorms?,[Transportation: There are buses and minibuses...,There are buses and minibuses operating betwee...,"The PC Rooms are open from 08:00 to 20:00, Mon...",[FAQ Question: What are the PC Rooms Working H...
4,Can students from outside Ankara apply for dorms?,[Application for Dormitories: Students whose f...,Students whose family resides outside of Ankar...,Hello! I'd be happy to help you with your ques...,[Registration Procedures Answer: METU Ankara C...


In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity, NonLLMContextPrecisionWithReference
from ragas import evaluate

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas import EvaluationDataset

In [9]:
test_subset = test_subset[['user_input', 'reference', 'reference_contexts', 'response', 'retrieved_contexts']]

In [10]:
test_subset

,user_input,reference,reference_contexts,response,retrieved_contexts
0,How has the Turkish Standards Institution cont...,The Office of Dormitories obtained the TSE-EN ...,[General Information: There are 19 dormitory b...,PC Rooms Regulations are a set of rules that g...,[FAQ Answer: PC Rooms which are operated by ...
1,How does the Rectorate influence the dormitory...,"Every year, the Executive Board of Dormitories...",[General Information: There are 19 dormitory b...,To re-enroll to an undergraduate program at ME...,[Registration Procedures Question: What are th...
2,What are the operting hurs for transport?,There are buses and minibuses operating betwee...,[Transportation: There are buses and minibuses...,Hello! I'd be happy to help you with your ques...,[FAQ Answer: Brief Information on IT Service...
3,What hours buses run from city center to dorms?,There are buses and minibuses operating betwee...,[Transportation: There are buses and minibuses...,"The PC Rooms are open from 08:00 to 20:00, Mon...",[FAQ Question: What are the PC Rooms Working H...
4,Can students from outside Ankara apply for dorms?,Students whose family resides outside of Ankar...,[Application for Dormitories: Students whose f...,Hello! I'd be happy to help you with your ques...,[Registration Procedures Answer: METU Ankara C...


In [11]:
eval_dataset = EvaluationDataset.from_pandas(test_subset)

In [12]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI")
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

metrics = [
        ## RAG Triad ##
    ## Context Relevance
    LLMContextRecall(llm=evaluator_llm), 
    NonLLMContextPrecisionWithReference(),

    ## Faithfulness
    FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),

    ## Answer Relevance
    SemanticSimilarity(embeddings=evaluator_embeddings),
    
]

    # 1. Answer Relevance
    #     Metric: SemanticSimilarity
    #         This metric evaluates how semantically similar the generated answer is to the ground truth answer. 
    #         It ensures that the generated response is relevant to the user's query and aligns well with the expected answer.

    # 2. Context Relevance
    #     Metric: LLMContextRecall
    #         This measures the relevance of the retrieved context used to generate the answer. 
    #         It checks whether the retrieved documents or context actually support the generated answer.

    # 3. Faithfulness
    #     Metrics: FactualCorrectness and Faithfulness
    #         FactualCorrectness: Validates if the generated answer is factually correct based on the provided context, ensuring that no hallucinations occur.
    #         Faithfulness: Ensures that the answer is faithful to the retrieved context and does not introduce information not present or supported by the context.


results = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [13]:
results

{'context_recall': 0.4000, 'non_llm_context_precision_with_reference': 0.2000, 'factual_correctness': 0.0520, 'faithfulness': 0.3719, 'semantic_similarity': 0.8324}

In [15]:
results_df = results.to_pandas()

In [25]:
results_df_metrics_stats= results_df[['context_recall',
       'non_llm_context_precision_with_reference', 'factual_correctness',
       'faithfulness', 'semantic_similarity']].describe()

In [1]:
from meetu_test_eval import meetu_evaluate_metrics

2024-12-01 19:18:30.462101: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 19:18:30.470360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733069910.482094  329282 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733069910.485485  329282 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 19:18:30.497323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
a,b,c, d = meetu_evaluate_metrics(openai_api_key=os.getenv("OPEN_AI"), demo=True)

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [4]:
d

0.4042426586151123